# Notebook for developing python and testing

In [ ]:
import requests as r

import pandas as pd
import datetime as dt
from sqlalchemy import DateTime



In [ ]:


def get_yesterday():
    """
        Get yesterdays date as a string
    """
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    theday = dt.date.today() - dt.timedelta(days=1)
    day =  theday.day
    month = months[theday.month-1]

    year = theday.year
    if day < 10:
        day = f"0{str(day)}"
        
    return  f"{year}-{month}-{day}"


def get_yesterday_url():
    date_ref = get_yesterday()
    return f"https://api.weatherapi.com/v1/history.json?q=Perth&dt={date_ref}&end_dt={date_ref}&hour=8&key=96792ecea70641218aa32831252402"


In [ ]:
response = r.get(url=get_yesterday_url())
data_dict = json.loads(response.content)
del response

data = pd.json_normalize(data_dict['forecast']['forecastday'])




In [ ]:
#  this should be defined somewhere: 
data.columns = [
'date',
'date_epoch',
'hour',
'maxtemp_c',
'maxtemp_f',
'mintemp_c',
'mintemp_f',
'avgtemp_c',
'avgtemp_f',
'maxwind_mph',
'maxwind_kph',
'totalprecip_mm',
'totalprecip_in',
'totalsnow_cm',
'avgvis_km',
'avgvis_miles',
'avghumidity',
'daily_will_it_rain',
'daily_chance_of_rain',
'daily_will_it_snow',
'daily_chance_of_snow',
'condition_text',
'condition_icon',
'condition_code',
'uv',
'sunrise',
'sunset',
'moonrise',
'moonset',
'moon_phase',
'moon_illumination',
]

In [137]:
data.iloc[0, 2] = 8

In [138]:
data

,date,date_epoch,hour,maxtemp_c,maxtemp_f,mintemp_c,mintemp_f,avgtemp_c,avgtemp_f,maxwind_mph,...,condition_text,condition_icon,condition_code,uv,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,2025-02-25,1740441600,8,24.3,75.7,18.0,64.5,21.2,70.2,14.3,...,Sunny,//cdn.weatherapi.com/weather/64x64/day/113.png,1000,9.0,06:03 AM,06:56 PM,02:33 AM,05:20 PM,Waning Crescent,14


In [39]:
connection = "Server=joshwork\\MSSQLSERVER02;Database=db1;user=sa;password=password1;Trusted_Connection=True;TrustServerCertificate=True"


connection_string = "mssql+pyodbc://@joshwork\\MSSQLSERVER02/db1?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes&TrustServerCertificate=yes"



In [ ]:
data.iloc[0,0] = dt.date.today() - dt.timedelta(days=1)

In [ ]:

def clean_string_times(timestring):
    """
        Returns hundred hour int of a time string

    Args:
        timestring (string): string in formate '00:00 AM/PM'

    Returns:
        int: 100 hour int of time string, if incorrect format will return 0, needs logging.
    """
    try:
        time = timestring.split(" ")[0]
    except:
        return 0
    if timestring.split(" ")[1][0].lower() == "p": # case sensitivity removed
        return (  ((int(time.split(":")[0])+ 12)*100)   + int(time.split(":")[1])) 
    else:
        return (  ((int(time.split(":")[0]))*100)   + int(time.split(":")[1])) 



x = 25
for val in data.iloc[0,25:-2]:
    data.iloc[0, x] =   clean_string_times(val)
    x +=1
 

In [ ]:
data.to_sql( 
    name="WeatherDataModels",
    con=connection_string, 
    if_exists="append",
    index=False,
    dtype={'date' : DateTime()}
    )

1